![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [26]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/research/overview]
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf, grangercausalitytests

qb = QuantBook()
window = 365

In [27]:
    #Dickey-Fuller test stolen from https://www.hackdeploy.com/augmented-dickey-fuller-test-in-python/
    def ADF_Stationarity_Test(timeseries, significance_level=0.05, printResults = True):
        is_stationary = None
        
        adf_test = adfuller(timeseries, autolag='AIC')
        pValue = adf_test[1]
        
        if (pValue<significance_level):
            is_stationary = True
        else:
            is_stationary = False
        
        if printResults:
            dfResults = pd.Series(adf_test[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
            #Add Critical Values
            for key,value in adf_test[4].items():
                dfResults['Critical Value (%s)'%key] = value
            print('Augmented Dickey-Fuller Test Results:')
            print(dfResults)
            print('Is it stationary? '+str(is_stationary))

In [12]:
forex = qb.AddForex("USDCAD", Resolution.Daily, Market.Oanda).Symbol
forex_data = qb.History(forex, window)
print('USDCAD:')
forex_data['close'].plot()

In [13]:
oil = qb.AddCfd("WTICOUSD", Resolution.Daily, Market.Oanda).Symbol
oil_data = qb.History(oil, window)
plt.gca().invert_yaxis()
print('Crude Oil Inverted:')
oil_data['close'].plot()

In [14]:
print('Running Augmented Dickey-Fuller Test for Crude Oil...')
print()
ADF_Stationarity_Test(oil_data['close'].values)

In [15]:
print('Running Augmented Dickey-Fuller Test for Forex...')
print()
ADF_Stationarity_Test(forex_data['close'].values)

In [16]:
# We need to split the series into training and testing data
n_obs=70
X_train_oil, X_test_oil = oil_data['close'][0:-n_obs], oil_data['close'][-n_obs:]
#print(X_train_oil.shape, X_test_oil.shape)

In [17]:
# We need to split the series into training and testing data
n_obs=70
X_train_forex, X_test_forex = forex_data['close'][0:-n_obs], oil_data['close'][-n_obs:]
print(X_train_forex.shape, X_test_forex.shape)

In [18]:
# Difference the training data to make it stationary
train_oil = X_train_oil.dropna().values.tolist()
train_forex = X_train_forex.dropna().values.tolist()
#stationary_forex_data = X_train_forex.diff().dropna()

In [19]:
# create a differenced series
def difference(series, interval=1):
    diff = []
    for i in range(interval, len(series)):
        value = series[i] - series[i - interval]
        diff.append(value)
    return diff


In [20]:
#stationary_oil_data = pd.DataFrame(difference(oil_data['close'].values.tolist()))
#stationary_forex_data = pd.DataFrame(difference(forex_data['close'].values.tolist()))
stationary_oil_data = pd.DataFrame(difference(train_oil))
stationary_forex_data = pd.DataFrame(difference(train_forex))
print('Stationary Crude Oil:')
stationary_oil_data.plot()

In [21]:
print('Stationary Forex:')
stationary_forex_data.plot()

In [22]:
print('Running Augmented Dickey-Fuller Test for Stationary Crude Oil...')
print()
ADF_Stationarity_Test(stationary_oil_data.values)

In [23]:
print('Running Augmented Dickey-Fuller Test for Stationary Crude Oil...')
print()
ADF_Stationarity_Test(stationary_forex_data.values)

In [24]:
oil_cad = pd.DataFrame({'Oil': np.squeeze(stationary_oil_data),
                        'Cad': np.squeeze(stationary_forex_data)})

print('Stationary Oil vs Cad')
oil_cad.head()


In [25]:
# https://medium.com/swlh/using-granger-causality-test-to-know-if-one-time-series-is-impacting-in-predicting-another-6285b9fd2d1c

ganger_test = grangercausalitytests(oil_cad, maxlag=15)

In [ ]:
"""
Python Implementations of ARIMAX:
https://pyflux.readthedocs.io/en/latest/arimax.html
http://barnesanalytics.com/analyzing-multivariate-time-series-using-arimax-in-python-with-statsmodels
https://www.kaggle.com/viridisquotient/arimax
"""